# Chapter 13: Numpy

One of the main reasons Python is so popular is that a vast array of tools easily available and usable directly in the language.  This and the following chapters introduce some key some mathematical packages useful for data science and engineering.  Each tool has its own extensive online documentation, and the goal of these chapters is just to motivate you to explore and use these packages.  These chapters are also Jupyter notebooks, so if you load them with VSCode you can run the code snippets directly in the document.

**Numpy** is mathematical package that supports multidimensional arrays and operations on them such as matrix multiplication and linear algebra, see: [https://numpy.org/doc/](https://numpy.org/doc/).  All of the other packages described later rely on Numpy.  It is important to be familiar with multidimensional arrays and various ways they can be manipulated and accessed.

## Installation and Help

It is best to install these tools in a virtual environment, see [Chapter 12](Chapter_12_Virtual_Environments.md).  First **activate** your virtual environment and then, from the shell command line, install them with
```
python -m pip install numpy scipy matplotlib opencv-python
```
If a package is already installed, this will simply skip it.  Also, any packages that are prerequisites will be installed. 

Now, after the packages are installed they can be accessed by running Python and using the `import` command as explained in [Chapter 11: Modules and Packages](Chapter_11_Modules_and_Packages.md).  For example, Numpy is usually imported with

In [1]:
import numpy as np

Besides online documentation, you can get documentation in your interactive terminal with the `help()` function.  For example if you want to know how the numpy cross product works, you would type: `help(np.cross)`. 

## Numpy Arrays

Arrays provide significant computational efficiencies over lists.  Consider various ways to represent 3D points, such as those collected from a lidar.  A single point could be represented as a class with a `.x`, `.y`, `.z`, or as a list with 3 elements or as a 1D numpy array with 3 elements.  A point cloud could then be either a list of points or a 2D numpy array which stacks 1D point arrays.  Performing an operation over all the points, such as rotating them, could be done by iterating over the list or else as a matrix multiplication applied to the array.  Both are equivalent mathematically, and lists are more flexible, but array operations are faster.  For operations on just a few points (10s or so), which representation you use does not make too much difference.  When there are 10,000 or 100,000 points, which could be a single lidar scan, then array operations could be orders of magnitude faster.  In addition, Numpy and other packages supply operations that are very efficient over arrays and much slower over lists.

Let's do a simple example to compare a list operation with an array operation.  First, represent a collection of four 3D points as a list:

In [2]:
point_list = [[1.,0.,0.],[1.,2.,1.],[3.,0.,1.],[4.,2.,0.]]
point_list

[[1.0, 0.0, 0.0], [1.0, 2.0, 1.0], [3.0, 0.0, 1.0], [4.0, 2.0, 0.0]]

To add `1` to each element you can use list comprehension:

In [3]:
new_point_list = [ [x[0]+1,x[1]+1,x[2]+1] for x in point_list]
new_point_list

[[2.0, 1.0, 1.0], [2.0, 3.0, 2.0], [4.0, 1.0, 2.0], [5.0, 3.0, 1.0]]

Clearly that is a little awkward, besides being slow for long lists.  

Now lets convert our list of lists to a Numpy array:

In [4]:
point_array = np.array(point_list)
point_array

array([[1., 0., 0.],
       [1., 2., 1.],
       [3., 0., 1.],
       [4., 2., 0.]])

Our points have been stored as a 2D array with each *row* being a point and stacked vertically.  Now let's add `1` to each element of each point:

In [5]:
new_point_array = point_array + 1
new_point_array

array([[2., 1., 1.],
       [2., 3., 2.],
       [4., 1., 2.],
       [5., 3., 1.]])

Much easier eh?  And faster to boot!

## Indexing Numpy Arrays

The same indexing and slicing that we saw used for strings is available for each dimension of Numpy arrays.  The dimensions, or *axes* as they are called in Numpy, are numbered from left to right like this:  
```
array[<axis 0>, <axis 1>, ..., <axis N>]
```
For two-dimensional arrays that means **axis 0** specifies the row and **axis 1** specifies the column.  In our example, each point is a row, and we can obtain the first point like this:

In [6]:
new_point_array[0,:]             # All elements of row 0

array([2., 1., 1.])

Note, that it is possible to drop trailing colons `:`, although not leading colons, and so row 0 can also be specified simply as:

In [7]:
new_point_array[0]

array([2., 1., 1.])

Now the *x* values of all points are in column 0, and these can be obtained by slicing along axis 0:

In [8]:
new_point_array[:,0]           # A slice along axis 0 at index 0 of axis 1

array([2., 2., 4., 5.])

Notice that a single row is returned as a 1D array.  There are many more details to learn about indexing and adding dimensions to array which you can find in the official documentation: [https://numpy.org/doc/stable/reference/arrays.indexing.html](https://numpy.org/doc/stable/reference/arrays.indexing.html).  I strongly recommend you review this page. 

To get some practice working with axes, try using `np.concatentate()` to combine arrays along different dimensions.  Predict what you get when you do the following commands, and then try them:

In [9]:
np.concatenate( (point_array, point_array), axis=0)

array([[1., 0., 0.],
       [1., 2., 1.],
       [3., 0., 1.],
       [4., 2., 0.],
       [1., 0., 0.],
       [1., 2., 1.],
       [3., 0., 1.],
       [4., 2., 0.]])

In [10]:
np.concatenate( (point_array, point_array), axis=1)

array([[1., 0., 0., 1., 0., 0.],
       [1., 2., 1., 1., 2., 1.],
       [3., 0., 1., 3., 0., 1.],
       [4., 2., 0., 4., 2., 0.]])

## Cross Products with Numpy
Cross products are simple to perform with Numpy.  A 3-vector is represented with a length 3 one-dimensional Numpy array, and the cross product like this:

In [11]:
x = np.array([1.,0.,0.])
y = np.array([0.,1.,0.])
np.cross(x,y)

array([0., 0., 1.])

As one can confirm with the right-hand rule, the output is a unit vector on the `z` axis.  And to make life even easier, if you pass in length-3 lists, the `cross()` function will first convert them to Numpy arrays and then perform the cross product like this:

In [12]:
np.cross([1.,0.,0.],[0.,1.,0.])

array([0., 0., 1.])

Now say you want to take the cross product of `[1.,0.,0.]` with multiple vectors.  It is inefficient to write each product out on a separate line.  It is more succinct and faster to stack the vectors into a single array and then use a single `cross()` command to do all the cross products at once.  Thus first stack vectors:

In [13]:
vecs = np.array([[1.,0.,0.],[0.,1.,0.],[0.,-1.,0.],[0.,0.,1.]])
vecs

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0., -1.,  0.],
       [ 0.,  0.,  1.]])

Now do all the cross products with one command:

In [14]:

np.cross([1.,0.,0.], vecs)

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0., -1.],
       [ 0., -1.,  0.]])

## Broadcasting with Numpy Arrays

Our final topic is that of broadcasting in Numpy.  This enables efficient repetitive operations on arrays.  But before describing it, let's review elementwise operations.  Start with two arrays of points:

In [15]:
points = np.array([[1.,0.,0.],[1.,2.,1.],[3.,0.,1.],[4.,2.,0.]])
points

array([[1., 0., 0.],
       [1., 2., 1.],
       [3., 0., 1.],
       [4., 2., 0.]])

In [16]:
opoints = np.flip(points,axis=0)
opoints

array([[4., 2., 0.],
       [3., 0., 1.],
       [1., 2., 1.],
       [1., 0., 0.]])

If we add or multiply `points` and `opoints`, these operations are done elementwise.  For example:

In [17]:
spoints = points + opoints
spoints

array([[5., 2., 0.],
       [4., 2., 2.],
       [4., 2., 2.],
       [5., 2., 0.]])

For this to operate elementwise, it is important that `points` and `opoints` are the same shape.  Now, let's say we want to translate `spoints` by a single 3-vector:

In [18]:
t = np.array([1.,4.,9.]) 

How might we add `t` to `spoints` so that it is added to each 3D point?  

One way would be to stack 4 copies of `t` along `axis=0` to create a new `4 X 3` matrix and then do an elementwise addition.  This works, but is tedious, and could be inefficient depending how it is done.  A better way is through **broadcasting**.  

I will describe broadcasting for adding two rank `N` arrays, `A` and `B`. (The same principles apply to other elementwise operations like multiplication and division.)  To add `A` and `B`: 
* The size of each corresponding dimension must be the same 

**OR** 
* If any dimension sizes differ, then one of the corresponding sizes must be 1.  

Any dimension of `A` or `B` that is 1 causes the array to be replicated along that dimension to equal the size of the other. 

Let's look at an example

In [19]:
A = np.eye(4)                  # 4x4 identity matrix
A

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [20]:
B = 2*np.arange(3)[:,None] 
B

array([[0],
       [2],
       [4]])

The `[:,None]` adds a new dimension along axis 1 and turns shape of B from `(3,) --> (3,1)`.  Now let's add these arrays:

In [21]:
C = A + B
C

ValueError: operands could not be broadcast together with shapes (4,4) (3,1) 

To see what happened, let's compare the shapes of `A`: `(3,3)`, and `B`: `(3,1)`. Along axis 0, `A` and `B` have the same size.   Along axis 1 `A` has size 3 while `B` has size 1.  This means `B` will be replicated 3 times along axis 1.  So the following operation that will give the same result as broadcasting is:

In [ ]:
C = A + np.repeat(B,3,axis=1)         # Explicitly replicate along axis 1
C

array([[1., 0., 0.],
       [2., 3., 2.],
       [4., 4., 5.]])

Using broadcasting this can be done more succinctly.   And broadcasting works for arrays of any number of dimensions.  

There is one more detail.  If `A` or `B` has fewer dimensions than the other, then additional **leading** dimensions of size 1 are added so that both arrays have the same number of dimensions.  Let's consider some examples and whether `A` and `B` can be broadcast to create `C`:

| `A.shape` | `B.shape` | `C.shape` or invalid |
| --- |--- | --- |
| `(6,4,2)` | `(6,1,2)` | `(6,4,2)` |
| `(4,2,5)` | `(4,3,5)` | invalid [Dimension 1 has a `2` and `3`] |
| `(3,1,2,6)` | `(3,5,2,1)` | `(3,5,2,6)` |
| `(5,4)` | `(3,5,1)` | `(3,5,4)` [A leading 1 is added to `(5,4) --> (1,5,4)`] |
| `(4,5)` | `(4,5,1)` | invalid [Adding a leading dimension to `A` gives size `(1,4,5)`]

Now let's return to our task of translating `spoints` with `t`.  

In [ ]:
spoints = np.array([[5., 2., 0.], [4., 2., 2.], [4., 2., 2.], [5., 2., 0.]])
spoints

array([[5., 2., 0.],
       [4., 2., 2.],
       [4., 2., 2.],
       [5., 2., 0.]])

In [ ]:
t = np.array([1.,4.,9.]) 
t

array([1., 4., 9.])

It should now be clear what we can simply add them:

In [ ]:
npoints = spoints + t
npoints

array([[ 6.,  6.,  9.],
       [ 5.,  6., 11.],
       [ 5.,  6., 11.],
       [ 6.,  6.,  9.]])

What is happening here?  `spoints` is size `(4,3)` while `t` is size `(3,)`.  During broadcasting, `t` is first converted to size `(1,3)`, and then replicated four times along dimension 0 and then added.  Thus `t` is added to each of the four 3D points, effectively translating them by `t`. 

Here are some practice examples.  Predict what you will get from the following operations and then try them out:

In [ ]:
np.ones( (3,1) ) * np.arange(4)


array([[0., 1., 2., 3.],
       [0., 1., 2., 3.],
       [0., 1., 2., 3.]])

In [ ]:
np.ones( (2,4,3), dtype=np.int32 ) * np.array( [3,4,8], dtype=np.int32 )

array([[[2, 4, 8],
        [2, 4, 8],
        [2, 4, 8],
        [2, 4, 8]],

       [[2, 4, 8],
        [2, 4, 8],
        [2, 4, 8],
        [2, 4, 8]]])

Notice how `np.ones()` can create an array of any dimensionality, as specified by the first argument.  Also note that we can explicitly set the data type of an array when we create it, in this case as 32-bit integers.

___
## Exercises

Numpy arrays enable you to perform many operations directly on arrays without iterating.  This is important as iterating in Python is slow.  The following exercises should all be done without any iteration.

1. In an interactive Python environment, create a `6x8` 2D array called `img` in which each element value is its index (see below).  You can do this using the `np.arange()` function that is just like Python's `range()` function except that it returns a 1D numpy array.  Then you can reshape it into the size of your target array using `.reshape( (H,W) )`.  The argument to this is a tuple with the height and width.  You should be able to do all this in one line of code and then your `img` for a `6x8` array should look like:
```python
array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31],
       [32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47]])
```
2. In one Python line of code, use integer division to transform `img` into an array called `row` with each element equal to its row index, like this:
```python
array([[0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4, 4, 4, 4],
       [5, 5, 5, 5, 5, 5, 5, 5]], dtype=int32)
```
3. Similarly use one Python line of code to transform `img` into an array called `col` with each element equal to its column index, like this:
```python
array([[0, 1, 2, 3, 4, 5, 6, 7],
       [0, 1, 2, 3, 4, 5, 6, 7],
       [0, 1, 2, 3, 4, 5, 6, 7],
       [0, 1, 2, 3, 4, 5, 6, 7],
       [0, 1, 2, 3, 4, 5, 6, 7],
       [0, 1, 2, 3, 4, 5, 6, 7]], dtype=int32)
```
4. Use simple math operations applied to `row` and `col` to calculate a distance array called `dist` that gives the Euclidean distance to each element from the top-left element.   You can use Pythagorous theorem for this: `c = np.sqrt(a*a+b*b)`. To display it in an easily viewable 3-decimal point format use the following commands:
```python
>>> np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
>>> dist
array([[ 0.000,  1.000,  2.000,  3.000,  4.000,  5.000,  6.000,  7.000],
       [ 1.000,  1.414,  2.236,  3.162,  4.123,  5.099,  6.083,  7.071],
       [ 2.000,  2.236,  2.828,  3.606,  4.472,  5.385,  6.325,  7.280],
       [ 3.000,  3.162,  3.606,  4.243,  5.000,  5.831,  6.708,  7.616],
       [ 4.000,  4.123,  4.472,  5.000,  5.657,  6.403,  7.211,  8.062],
       [ 5.000,  5.099,  5.385,  5.831,  6.403,  7.071,  7.810,  8.602]])
>>> np.set_printoptions(formatter={}) # Undo the format option for future display
```
5.  Find the [Manhatten distance or Taxicab distance](https://en.wikipedia.org/wiki/Taxicab_geometry) from each element of `img` to the center of the center point `(2.5,3.5)`.  This is simply the absolute row difference plus the absolute column difference of each element from the center point.  The `abs()` function will help you here.  You should get a result like this:
```python
array([[6., 5., 4., 3., 3., 4., 5., 6.],
       [5., 4., 3., 2., 2., 3., 4., 5.],
       [4., 3., 2., 1., 1., 2., 3., 4.],
       [4., 3., 2., 1., 1., 2., 3., 4.],
       [5., 4., 3., 2., 2., 3., 4., 5.],
       [6., 5., 4., 3., 3., 4., 5., 6.]])
```
6.  Given two 1D arrays: `vec1 = np.array([1,2,3])`, and `vec2 = np.array([2,2,1,1])`, use the muliplcation operator in one line of code to calculate the outer product of these.  You'll need to transform `vec1` into a column vector before multiplying, and the answer you should get is:
```python
array([[2, 2, 1, 1],
       [4, 4, 2, 2],
       [6, 6, 3, 3]])
```
7. Use the commands `np.zeros()`, `np.eye()`, `np.concatenate()` and `.astype()` in one line of code to create the following matrix:
```python
array([[0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)
```   

___
### [Outline](../README.md), Next: [Chapter 14: Scipy](Chapter_14_Scipy.ipynb)